In [0]:

dbutils.widgets.text("load_date", "")
load_date = dbutils.widgets.get("load_date")
print(f"SILVER load_date = {load_date}")


In [0]:
from pyspark.sql.functions import col, upper, trim, regexp_replace, when, to_date, date_format, expr
from delta.tables import DeltaTable
from pyspark.sql.functions import current_timestamp, lit

In [0]:
"""
Cleans and normalizes bronze Delta tables for employees, departments, and jobs,
then writes the processed data to silver Delta tables in the specified Volumes paths.

- Employees: Trims and sanitizes names, fills missing emails, normalizes date_of_birth.
- Departments: Trims and sanitizes department names.
- Jobs: Trims and sanitizes job titles, normalizes start/end dates.

Output: Silver Delta tables for employees, departments, and jobs.
"""

df_emp_sl = spark.sql(
    f"""
    SELECT
      employee_id,
      first_name,
      last_name,
      date_of_birth,
      dept_code,
      email
    FROM edl_hc_datamart.bronze.employees
    WHERE _meta_load_date = '{load_date}'
    """
).withColumn("first_name", regexp_replace(trim(col("first_name")), r"[^A-Za-z0-9 ]", "_")
).withColumn("last_name",  regexp_replace(trim(col("last_name")),  r"[^A-Za-z0-9 ]", "_")
).withColumn("email", when(col("email").isNull() | (trim(col("email"))==""), "unknown@example.com")
                     .otherwise(trim(col("email")))
).withColumn("date_of_birth", to_date(col("date_of_birth"))
).withColumn("date_of_birth_fmt", date_format(col("date_of_birth"), "dd-MMM-yyyy")
).select(
    "employee_id", "first_name", "last_name",
    "date_of_birth", "date_of_birth_fmt",
    "dept_code", "email"
)

# Load target table
target = DeltaTable.forName(spark, "edl_hc_datamart.silver.employees")

# Prepare source DataFrame with SCD columns
df_emp_scd = df_emp_sl.withColumn("is_active", expr("true")) \
                     .withColumn("effective_date", current_timestamp()) \
                     .withColumn("end_date", lit('1900-01-01').cast('timestamp'))
# Mark old records as inactive
target.alias("tgt").merge(
    df_emp_scd.alias("src"),
    "tgt.employee_id = src.employee_id AND tgt.is_active = true"
).whenMatchedUpdate(set={
    "is_active": "false",
    "end_date": "current_timestamp()"
}).whenNotMatchedInsertAll().execute()

df_emp = df_emp_scd.write.format("delta").mode("overwrite").saveAsTable("edl_hc_datamart.silver.employees")

In [0]:
# display(df_emp)

In [0]:
# display(df_emp)

In [0]:
# Departments

df_dept_sl = spark.sql(
    f"""
    SELECT
      regexp_replace(
          regexp_replace(trim(dept_name), '[^A-Za-z0-9 ]', '_'),
          '_+',
          '_'
      ) AS dept_name,
      -- Normalize dept_code: trim + uppercase for consistency
      upper(trim(dept_code)) AS dept_code
    FROM edl_hc_datamart.bronze.departments
    WHERE _meta_load_date = '{load_date}'
    """
).select("dept_name", "dept_code")


# Load target table
target_dept = DeltaTable.forName(spark, "edl_hc_datamart.silver.departments")

# Prepare source DataFrame with SCD columns
df_dept_scd = df_dept_sl.withColumn("is_active", expr("true")) \
                        .withColumn("effective_date", current_timestamp()) \
                        .withColumn("end_date", lit('1900-01-01').cast('timestamp'))

# Mark old records as inactive
target_dept.alias("tgt").merge(
    df_dept_scd.alias("src"),
    "tgt.dept_code = src.dept_code AND tgt.is_active = true"
).whenMatchedUpdate(set={
    "is_active": "false",
    "end_date": "current_timestamp()"
}).whenNotMatchedInsertAll().execute()

df_dept_scd.write.format("delta").mode("overwrite").saveAsTable("edl_hc_datamart.silver.departments")

In [0]:
# Jobs
df_jobs_sl = spark.sql(
    f"""
    SELECT
      employee_id,
      end_date,
      job_title,
      position_id,
      start_date,
      status,
      salary_amount,
      salary_currency,
      salary_frequency,
      salary_effective_from,
      salary_effective_to
    FROM edl_hc_datamart.bronze.jobs
    WHERE _meta_load_date = '{load_date}'
    """
).withColumn("job_title",
    regexp_replace(trim(col("job_title")), r"[^A-Za-z0-9 ]", "_")
).withColumn("start_date", to_date(col("start_date")))\
.withColumn("end_date",   to_date(col("end_date")))\
.withColumn("start_date_fmt", date_format(col("start_date"), "dd-MMM-yyyy"))\
.withColumn("end_date_fmt",
    when(col("end_date").isNotNull(), date_format(col("end_date"), "dd-MMM-yyyy"))
).withColumn("salary_currency", upper(trim(col("salary_currency")))).withColumn("salary_frequency", upper(trim(col("salary_frequency")))).select(
    "employee_id",
    "position_id",
    "job_title",
    "status",
    "start_date", "start_date_fmt",
    "end_date",   "end_date_fmt",
    "salary_amount",
    "salary_currency",
    "salary_frequency",
    "salary_effective_from",
    "salary_effective_to"
)

# Load target table
target_jobs = DeltaTable.forName(spark, "edl_hc_datamart.silver.jobs")

df_jobs_scd = df_jobs_sl.withColumn("employee_id", col("employee_id").cast("int")) \
                        .withColumn("salary_amount", col("salary_amount").cast("int")) \
                        .withColumn("is_active", expr("true")) \
                        .withColumn("effective_date", current_timestamp()) \
                        .withColumn("effective_end_date", lit('1900-01-01').cast('timestamp'))

# Mark old records as inactive and new records as active during insert
target_jobs.alias("tgt").merge(
    df_jobs_scd.alias("src"),
    "tgt.employee_id = src.employee_id AND tgt.position_id = src.position_id AND tgt.is_active = true"
).whenMatchedUpdate(set={
    "is_active": "false",
    "effective_end_date": "current_timestamp()"
}).whenNotMatchedInsertAll().execute()

df_jobs_scd.write.format("delta").mode("overwrite").saveAsTable("edl_hc_datamart.silver.jobs")


In [0]:
"""
Cleans and normalizes the bronze Delta attendance table, then writes the processed data to the silver Delta table with SCD logic.

- Renames columns for consistency.
- Trims and sanitizes attendance_date.
- Normalizes attendance_date to date type and formats as dd-MMM-yyyy.
- Standardizes present to boolean, handling various representations.
- Fills missing present values as False (absent).
- Adds SCD columns: is_active, effective_date, end_date.

Output: Silver Delta table for attendance.
"""

# Data cleaning and transformation

df_att_sl = spark.sql(
    f"""
    SELECT
      employee_id,
      present,
      date
    FROM edl_hc_datamart.bronze.attendance
    WHERE _meta_load_date = '{load_date}'
    """
).withColumn("attendance_date", to_date(regexp_replace(col("date"), r"[^0-9\-]", "")))\
.withColumn("attendance_date_fmt", date_format(col("attendance_date"), "dd-MMM-yyyy"))\
.drop("date")\
.withColumn(
    "present",
    when(col("present").cast("string").isin("TRUE","True","true","1","Y"), True)
    .when(col("present").cast("string").isin("FALSE","False","false","0","N"), False)
    .otherwise(expr("try_cast(present as boolean)"))
)\
.withColumn("present", when(col("present").isNull(), False).otherwise(col("present")))\
.select("employee_id", "attendance_date", "attendance_date_fmt", "present")


# Load target table
target_att = DeltaTable.forName(spark, "edl_hc_datamart.silver.attendance")

# Prepare source DataFrame with SCD columns
df_att_scd = df_att_sl.withColumn("is_active", expr("true")) \
                      .withColumn("effective_date", current_timestamp()) \
                      .withColumn("end_date", lit('1900-01-01').cast('timestamp'))

# Mark old records as inactive
target_att.alias("tgt").merge(
    df_att_scd.alias("src"),
    "tgt.employee_id = src.employee_id AND tgt.attendance_date = src.attendance_date AND tgt.is_active = true"
).whenMatchedUpdate(set={
    "is_active": "false",
    "end_date": "current_timestamp()"
}).whenNotMatchedInsertAll().execute()

df_att_scd.write.format("delta").mode("overwrite").saveAsTable("edl_hc_datamart.silver.attendance")



In [0]:
# display(df_att_sl)

In [0]:
# dbutils.fs.ls("/FileStore")